# Projet spécialité Data-IA

# Analyse de données : l'industrie automobile et les changements climatiques

In [1]:
print("hello")

hello


Scrapper les données mondiales des ventes/émissions de CO2 des véhicules terrestre motorisés dans le monde, puis par continent, puis par pays de l'Europe, et pour une personne

# Scrapping des données mondiales

In [2]:
import requests # requêter sur le site
from bs4 import BeautifulSoup # parcourir le HTML
import pandas as pd # formatter les données/exporter en CSV

## Les ventes de véhicules, par années/pays/continent de 1999 à 2022

Nous avons trouvé le site de OICA. Une rubrique répertorie la production de véhicules motorisés dans le monde depuis 1999.

L'idée est donc de scraper ces données, dans la catégorie <b>production-statistics/{ANNÉE}-statistics</b>

Nous travaillerons depuis 2015

In [41]:
# CTRL + '/' pour commenter plusieurs lignes à la fois

import datetime

def get_current_year() -> int: 
    """ 
    for automatisation of the dataset. In 2025, 
    the program will research for the 2025's web page
    """
    today = datetime.date.today()
    year = today.year
    print(year)

    return year

def scrape_page(soup: BeautifulSoup, world_production_list: list):
    table = soup.find("tbody")
    countries = table.find_all("tr")

    # print(countries)

    for country_tr in countries:
        country_name = country_tr.find("td", {"class": f"column-1"}).text
        #print(country_tr)
        print(country_name)

    # for quote_div in quotes:
    #     quote = quote_div.find("span", {"class": "text"}).text
    #     author = quote_div.find("small", {'class': 'author'}).text
    #     tags = quote_div.find_all("a", {'class': 'tag'})
        
    #     dict = {
    #         'quote': '',
    #         'author': '',
    #         'tags': '',
    #     }

    #     dict['quote'] = quote[1:-1]
    #     dict['author'] = author
    #     tag_list = []
    #     for tag in tags:
    #         tag_list.append(tag.text)
    #     dict['tags'] = tag_list            

    #     quote_list.append(dict)

    # next = soup.find("li", class_='next') # deuxième manière de faire un class=
    # if next != None:
    #     print('Accès à la page suivante')
    #     next_url = next.find("a").get('href')
    #     req = requests.get(target +  next_url)
    #     next_soup = BeautifulSoup(req.text, "html.parser")
    #     scrape_page(next_soup, quote_list)
    # print('Aucune autre page')

begin_year = 2015
end_year = get_current_year()

target = f'https://www.oica.net/category/production-statistics/{begin_year}-statistics'
# https://www.oica.net/category/production-statistics/2015-statistics/
req = requests.get(target)

if req.status_code != 200:
    print("Erreur")
    exit(0)
else:
    print(f"Accès au site {target}, status = {req.status_code}")

soup = BeautifulSoup(req.text, "html.parser")

if __name__ == "__main__":
    print(f"Date de début {begin_year}; Date de fin : {end_year}")

    world_production_list = []
    # print(world_production_list)
    scrape_page(soup, world_production_list)
    # print(f"Final list of quotes {quote_list}")
    # df = pd.DataFrame(quote_list)
    # df.insert(0, 'ID', range(1, len(quote_list) +1))
    # df.to_csv('quote_list.csv', index=False, sep=';')


2024
Accès au site https://www.oica.net/category/production-statistics/2015-statistics, status = 200
Date de début 2015; Date de fin : 2024
Argentina
Australia
Austria
Belgium
Brazil
Canada
China
Czech Rep.
Egypt
Finland
France
Germany
Hungary
India
Indonesia
Iran
Italy
Japan
Malaysia
Mexico
Netherlands
Poland
Portugal
Romania
Russia
Serbia
Slovakia
Slovenia
South Africa
South Korea
Spain
Sweden
Taiwan
Thailand
Turkey
Ukraine
UK
USA
Uzbekistan
Others
Total
